# ML-Lecture2 with Random Forests with in-class Notes

Note, i had to 

    $source activate fastai 

to make sure the library import works otherwise you might have opencv errors.

#### Loading the libraries from the first two blocks with random forests

In [1]:
%load_ext autoreload
%autoreload 2
# load libraries and edit the modules so you can use them


%matplotlib inline
# allows inline plotting

In [2]:
import sys
sys.path.append("/Users/tlee010/Desktop/github_repos/fastai/") # go to parent dir

from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

### Where does a particular python command it come from?

The left most term is the library where the command comes from 

In [3]:
display

<function IPython.core.display.display>

### what else can we learn about a command?

In [4]:
?display

### Source code for the Python?

In [5]:
??display

### What parameters does this python function take?

SHIFT + TAB

SHIFT + TAB x2 brings up documentation

SHIFT + TAB x3 new window with documentation

In [6]:
PATH = '/Users/tlee010/Desktop/github_repos/fastai/data/bulldozers'

## What is git ignore?

It's a hidden file that keeps track of files that you dont want to be replicated to the server. .gitignore. You can use wildcards to exclude large numbers of files. For instance tmp\* would ignore all the tmp prefixed folders. You can also put it in subdirectories.

This is really useful for ignoring:

- large datasets (don't want to post on github)
- credentials (keys)
- configuration files
- backup files
- scratch files

## What is symlink?

lets you alias folders if you want to point to another folder
    # documentation to create a symlink
    $man lm

## Let's get the data

https://www.kaggle.com/c/bluebook-for-bulldozers/data

Download (need to login) and unzip

If you don't have unzip:

```bash
$brew install unzip
$unzip file.zip
```

#### check the file location

In [7]:
!ls -l /Users/tlee010/kaggle/bulldozers/

total 246368
-rwxr-xr-x@ 1 tlee010  staff  116403970 Jan 24  2013 Train.csv
-rw-r--r--@ 1 tlee010  staff    9732240 Oct 26 14:20 Train.zip


#### Lets look at the system path

Add { } to force it to use bash PATH instead of python PATH

In [8]:
!ls {PATH}

ls: /Users/tlee010/Desktop/github_repos/fastai/data/bulldozers: No such file or directory


## How can you download straight to AWS? - Firefox trick

CTRL-SHIFT-I for developer console within Firefox

1. If you hit download, and track the actual link that is being used. 
2. Pause it
3. right click the download record, you should get a long curl string It has the cookies / auth details. You can paste in the AWS. Then need to add a '-o filename.zip'. That should download very quickly

### Bulldozers

Predicting the auction sale price for a piece of heavy equipment to create a blue blook for bulldozers.


The key fields are in train.csv are:

- SalesID: the uniue identifier of the sale
- MachineID: the unique identifier of a machine.  A machine can be sold multiple times
- saleprice: what the machine sold for at auction (only provided in train.csv)
- saledate: the date of the sale

### Look at the Data

In [9]:
!head /Users/tlee010/kaggle/bulldozers/Train.csv | head -3

In [10]:
%time !wc  /Users/tlee010/kaggle/bulldozers/Train.csv

  401126 8009543 116403970 /Users/tlee010/kaggle/bulldozers/Train.csv
CPU times: user 7.29 ms, sys: 18.1 ms, total: 25.4 ms
Wall time: 560 ms


### Using dataframes (Pandas)  - load takes a while
Added %time to track how long it takes to load the data

In [11]:
import pandas as pd

%time df_raw = pd.read_csv('/Users/tlee010/kaggle/bulldozers/Train.csv', low_memory=False, parse_dates=["saledate"])


CPU times: user 1min 9s, sys: 569 ms, total: 1min 10s
Wall time: 1min 10s


#### Quick note:

We added a `parse_dates` option to the read import to force data typing. For a full list of parameters and options, **check pandas documentation **:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html

### Look at the first few rows

In [12]:
df_raw.head(10)

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1139255,26500,1001274,4605,121,3.0,2004,508.0,Low,2008-12-18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1139256,21000,772701,1937,121,3.0,1993,11540.0,High,2004-08-26,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
7,1139261,27000,902002,3539,121,3.0,2001,4883.0,High,2005-11-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1139272,21500,1036251,36003,121,3.0,2008,302.0,Low,2009-08-27,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
9,1139275,65000,1016474,3883,121,3.0,1000,20700.0,Medium,2007-08-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional


### to see all columns ( custom function)

In [13]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)

In [14]:
display_all(df_raw.tail().transpose())

,401120,401121,401122,401123,401124
SalesID,6333336,6333337,6333338,6333341,6333342
SalePrice,10500,11000,11500,9000,7750
MachineID,1840702,1830472,1887659,1903570,1926965
ModelID,21439,21439,21439,21435,21435
datasource,149,149,149,149,149
auctioneerID,1,1,1,2,2
YearMade,2005,2005,2005,2005,2005
MachineHoursCurrentMeter,NaN,NaN,NaN,NaN,NaN
UsageBand,NaN,NaN,NaN,NaN,NaN
saledate,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00


In [15]:
display_all(df_raw.describe(include='all').transpose())

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
SalesID,401125,NaN,NaN,NaN,NaN,NaN,1.91971e+06,909021,1.13925e+06,1.41837e+06,1.63942e+06,2.24271e+06,6.33334e+06
SalePrice,401125,NaN,NaN,NaN,NaN,NaN,31099.7,23036.9,4750,14500,24000,40000,142000
MachineID,401125,NaN,NaN,NaN,NaN,NaN,1.2179e+06,440992,0,1.0887e+06,1.27949e+06,1.46807e+06,2.48633e+06
ModelID,401125,NaN,NaN,NaN,NaN,NaN,6889.7,6221.78,28,3259,4604,8724,37198
datasource,401125,NaN,NaN,NaN,NaN,NaN,134.666,8.96224,121,132,132,136,172
auctioneerID,380989,NaN,NaN,NaN,NaN,NaN,6.55604,16.9768,0,1,2,4,99
YearMade,401125,NaN,NaN,NaN,NaN,NaN,1899.16,291.797,1000,1985,1995,2000,2013
MachineHoursCurrentMeter,142765,NaN,NaN,NaN,NaN,NaN,3457.96,27590.3,0,0,0,3025,2.4833e+06
UsageBand,69639,3,Medium,33985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,401125,3919,2009-02-16 00:00:00,1932,1989-01-17 00:00:00,2011-12-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### .. now that EDA is complete, let's model and evaluation

From bulldozer Kaggle overview:

    The evaluation metric for this competition is the RMSLE (root mean squared log error) between the actual and predicted auction prices.

    Sample submission files can be downloaded from the data page. Submission files should be formatted as follows:

    Have a header: "SalesID,SalePrice"
    Contain two columns
    SalesID: SalesID for the validation set in sorted order
    SalePrice: Your predicted price of the sale
    
#### About metrics:

It's not always MSE, or RMSE, sometimes its profitability, its important to identify and know what the target metric is.


### Let's make it Log sale price

In [16]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

#### RandomForestRegressor - continous variable

#### RandomForestClassifier - identifying binary or multiclass categorical variables

RandomForest in general is trivially parallelizable. It's easy to distribute the work load for a cluster of machines. This is triggered by the parameter:

    njobs = -1

In [17]:
#SK LEARN works all teh same way with all algos

# start an ML object (it will start with defaults and empty)
m = RandomForestRegressor(n_jobs=-1)

# then send it the data so the model can be 'FIT'
m.fit(df_raw.drop('SalePrice', axis=1), df_raw.SalePrice)

ValueError: could not convert string to float: 'Conventional'

#### Stack Trace - traces the error through all the nested function calls.

### Check the bottom line for the true error ( 'could not convert string to float' - 'conventional')

This dataset contains a mix of **continuous** and **categorical** variables.

The following method extracts particular date fields from a complete datetime for the purpose of constructing categoricals.  You should always consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities.

In [18]:
add_datepart(df_raw, 'saledate')
df_raw.saleYear.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleYear, dtype: int64

The categorical variables are currently stored as strings, which is inefficient, and doesn't provide the numeric coding required for a random forest. Therefore we call `train_cats` to convert strings to pandas categories.

Random forest can handle categorical numbers that are numbers

In [19]:
??add_datepart

#### Let's look under the hood

```python
Signature: add_datepart(df, fldname)
Source:   
def add_datepart(df, fldname):
    fld = df[fldname]
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = (fld - fld.min()).dt.days
    df.drop(fldname, axis=1, inplace=True)
File:      ~/Desktop/github_repos/fastai/fastai/structured.py
Type:      function
```

Let's walk through the function code for this, it uses regex to find the fieldname, and it will replace the date column with multiple columns such as 'day','month','year','etc'. 

#### Are they useful? Include it, and every variant that might be useful. Max, Min, Mean. 

What about the curse of dimensionality - if you have too many columns vs. rows? In practice that doesn't happen. More date the better.

#### No Free Lunch Theorem

For all datasets, there is no one particularly better theorem. (this means all possible datasets, random or otherwise)

#### Free Lunch Theorem

In practice that random forests are the best techique for most cases, mainly because most real world problems are not made of random datasets. 

### Replace categories

In [20]:
??train_cats(df_raw)

```python
Signature: train_cats(df)
Source:   
def train_cats(df):
    for n,c in df.items():
        if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()
File:      ~/Desktop/github_repos/fastai/fastai/structured.py
Type:      function
```

In [21]:
train_cats(df_raw)

In [22]:
df_raw.UsageBand.cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

#### What do we do with missing values - lets take a look at the % null

In [49]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

## Save the data with feather format

In [50]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/raw')

In [52]:
??proc_df

```python
Signature: proc_df(df, y_fld, skip_flds=None, do_scale=False, preproc_fn=None, max_n_cat=None, subset=None)
Source:   
def proc_df(df, y_fld, skip_flds=None, do_scale=False,
            preproc_fn=None, max_n_cat=None, subset=None):
    if not skip_flds: skip_flds=[]
    if subset: df = get_sample(df,subset)
    df = df.copy()
    if preproc_fn: preproc_fn(df)
    y = df[y_fld].values
    df.drop(skip_flds+[y_fld], axis=1, inplace=True)

    for n,c in df.items(): fix_missing(df, c, n)
    if do_scale: mapper = scale_vars(df)
    for n,c in df.items(): numericalize(df, c, n, max_n_cat)
    res = [pd.get_dummies(df, dummy_na=True), y]
    if not do_scale: return res
    return res + [mapper]
File:      ~/Desktop/github_repos/fastai/fastai/structured.py
Type:      function
```

this will go through each column and run the `fix missing`

In [53]:
??fix_missing

```python
Signature: fix_missing(df, col, name)
Source:   
def fix_missing(df, col, name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum(): df[name+'_na'] = pd.isnull(col)
        df[name] = col.fillna(col.median())
File:      ~/Desktop/github_repos/fastai/fastai/structured.py
Type:      function

```

If it's numeric, create a new column that has the same tells you if missing or not. New column that where missing variables will be replaced with the median value for that continous field.


In [54]:
df, y = proc_df(df_raw, 'SalePrice')

In [56]:
df.columns

Index(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID',
       'YearMade', 'MachineHoursCurrentMeter', 'UsageBand', 'fiModelDesc',
       'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries', 'fiModelDescriptor',
       'ProductSize', 'fiProductClassDesc', 'state', 'ProductGroup',
       'ProductGroupDesc', 'Drive_System', 'Enclosure', 'Forks', 'Pad_Type',
       'Ride_Control', 'Stick', 'Transmission', 'Turbocharged',
       'Blade_Extension', 'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower',
       'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control',
       'Tire_Size', 'Coupler', 'Coupler_System', 'Grouser_Tracks',
       'Hydraulics_Flow', 'Track_Type', 'Undercarriage_Pad_Width',
       'Stick_Length', 'Thumb', 'Pattern_Changer', 'Grouser_Type',
       'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',
       'saleI

## Run your first RandomForest Model

In [57]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

0.98305857537865071

Is this overfitting?

### Setup Test and Train Split

In [58]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 12000  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((389125, 66), (389125,), (12000, 66))

In [59]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [60]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

CPU times: user 1min 17s, sys: 666 ms, total: 1min 17s
Wall time: 15.4 s
[0.09067208421281527, 0.2523758498048438, 0.98281767127977293, 0.88625205413504937]
